In [60]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [3]:
os.getcwd()

'/Users/alexrotondo/Documents/nlp'

In [8]:
with open('../../Downloads/JEOPARDY_QUESTIONS1.json', 'r') as file:
    questions = json.load(file)

In [10]:
questions[0]

{'category': 'HISTORY',
 'air_date': '2004-12-31',
 'question': "'For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory'",
 'value': '$200',
 'answer': 'Copernicus',
 'round': 'Jeopardy!',
 'show_number': '4680'}

In [29]:
def preprocess(questions):
    processedQuestions = []
    for question in questions:
        processedQuestions += ['<bos> ' + question['question'].lower()[1:-1] + ' <eos>']
    return processedQuestions

In [30]:
processedQuestions = preprocess(questions)

In [31]:
n = len(processedQuestions)

In [32]:
n

216930

In [33]:
processedQuestions[2]

'<bos> the city of yuma in this state has a record average of 4,055 hours of sunshine each year <eos>'

In [39]:
def get_token_counts(questions):
    tokenCounts = {}
    for question in questions:
        tokens = question.split(' ')
        for token in tokens:
            if token in tokenCounts:
                tokenCounts[token] += 1
            else:
                tokenCounts[token] = 1
    return tokenCounts

In [41]:
tokenCounts = get_token_counts(processedQuestions)

In [96]:
filteredTokenCounts = {'<UNK>':0}
for token, count in tokenCounts.items():
    if count >= 5:
        filteredTokenCounts[token] = count
    else:
        filteredTokenCounts['<UNK>'] += 1

In [68]:
tokensDf = pd.DataFrame()
tokensDf['tokens'] = tokenCounts.keys()
tokensDf['count'] = tokenCounts.values()

In [69]:
tokensDf.sort_values(by='count', ascending=False)

,tokens,count
0,<bos>,216930
18,<eos>,216930
2,the,169873
15,this,123074
6,of,112716
...,...,...
124777,ptarmigans,1
41933,flash-heated,1
41937,miles:,1
124774,typewriter's,1


In [70]:
tokensDf.iloc[80000, :]

tokens    href="http://www.j-archive.com/media/2008-10-1...
count                                                     1
Name: 80000, dtype: object

In [86]:
def filterTokens(tokens, tokenCounts=tokenCounts):
    filteredTokens = []
    for t in tokens:
        if tokenCounts[t] >= 10:
            filteredTokens += [t]
        else:
            filteredTokens += ['<UNK>']
    return filteredTokens

In [87]:
def get_n_grams(n, questions):
    ngrams = {}
    for question in questions:
        tokens = filterTokens(question.split(' '))
        grams = [tuple(tokens[i:i+n]) for i in range(len(tokens) - n)]
        for gram in grams:
            if gram in ngrams:
                ngrams[gram] += 1
            else:
                ngrams[gram] = 1
    return ngrams

In [88]:
bigrams = get_n_grams(2, processedQuestions)

In [89]:
bigramDf = pd.DataFrame()
bigramDf['bigram'] = bigrams.keys()
bigramDf['count'] = bigrams.values()

In [90]:
bigramDf.sort_values(by='count', ascending=False)

,bigram,count
73,"(<UNK>, <UNK>)",48770
33,"(the, <UNK>)",25439
70,"(of, the)",23140
134,"(<bos>, this)",22689
54,"(<bos>, in)",22234
...,...,...
341663,"(number, together,)",1
341661,"(use, measuring)",1
8392,"(5,, 1996,)",1
341658,"(line, over)",1


In [93]:
def get_n_gram_probabilities(ngrams, tokenCounts):
    for ngram in ngrams:
        ngrams[ngram] /= tokenCounts[ngram[0]]
    return ngrams

In [97]:
bigramProbabilities = get_n_gram_probabilities(bigrams, filteredTokenCounts)

In [103]:
import random

def get_possible_next_words(lastWord, bigrams=bigrams):
    return [bigram for bigram in bigrams.keys() if bigram[0] == lastWord]

def highest_probable_next_word(possibleBigrams, bigramProbabilities=bigramProbabilities):
    probabilities = [(bigramProbabilities[bigram], bigram) for bigram in possibleBigrams]
    return list(sorted(probabilities, reverse=True))[0][1][1]

def random_choice(possibleBigrams):
    return random.choice(possibleBigrams)[1]

def generate_jeopardy_question(n, bigramProbabilities=bigramProbabilities):
    nextWord = '<bos>'
    lastWord = nextWord
    question = ''
    
    while lastWord != '<eos>' and n > 0:
        
        nextWordOptions = get_possible_next_words(lastWord, bigrams)
        
        if lastWord == '<bos>':
            nextWord = random_choice(nextWordOptions)
        else:
            nextWord = highest_probable_next_word(nextWordOptions, bigramProbabilities)
        
        question += (' ' + nextWord)
        lastWord = nextWord
        n -= 1
    
    return question

In [104]:
q = generate_jeopardy_question(5)

In [105]:
print(q)

 tree is the <UNK> <UNK>
